In [ ]:
#!/usr/bin/env python
from __future__ import print_function

import actionlib
import rospy

from people_msgs.msg import PositionMeasurementArray, PositionMeasurement
from tf import TransformListener

from leg_tracker.msg import Person, PersonArray, Leg, LegArray

from move_base_msgs.msg import MoveBaseGoal, MoveBaseAction
from geometry_msgs.msg import Pose, PoseStamped, PoseWithCovarianceStamped

from tf_conversions import posemath

import numpy as np

from actionlib import SimpleActionClient

In [ ]:
face_tracked_topic = "face_tracked"
legs_tracked_topic = "people_tracked"
movebase_ns = "move_base"
max_match_distance = 2

In [ ]:
class Follower():
    def __init__(self):
        """Initialize Node
        Sets up the topic subscriptions, transform listener, and book keeping
        """
        rospy.init_node('walker', anonymous=True)
        
        self.tf = TransformListener()
        rospy.Subscriber(face_tracked_topic, PositionMeasurement, self.face_callback)
        rospy.Subscriber(legs_tracked_topic, PersonArray, self.legs_callback)
        
        self.goal_pup = rospy.Publisher("/move_base_simple/goal", PoseStamped, queue_size=10)

        self.face_tracked = PositionMeasurement()
        self.legs_tracked = PersonArray()
        self.person_tracked = Person()

        self.target_face_id = None
        self.target_legs_id = None
        rospy.spin()

    def face_callback(self, msg):
        """ Face topic callback
        Use new tracked face message to find matching legs
        """
        rospy.loginfo(rospy.get_caller_id() + " Recived Face ID: %s", str(msg.object_id))
        
        # Move face point to legs reference frame
        face_frame_to_leg_frame = self.tf.lookupTransformFull( # Get TF between face and leg frames
            self.legs_tracked.header.frame_id, # target_frame
            self.legs_tracked.header.stamp,    # target_time
            msg.header.frame_id,               # source_frame
            msg.header.stamp,                  # source_time
            'base_link')                       # fixed_frame
        face_frame_to_leg_frame = posemath.fromTf(face_frame_to_leg_frame)
        face_frame_to_leg_frame = posemath.toMatrix(face_frame_to_leg_frame)
        face_point = np.array([[msg.pos.x],[msg.pos.y],[msg.pos.z],[1]])
        face_point_fixed = (face_frame_to_leg_frame*face_point)[:,3][0:2] #3] # PointB = TF_A->B * PointA
        
        # Book keeping to search for closest pair of legs
        best_match_index = None
        best_match_distance = max_match_distance
        legs_tracked = self.legs_tracked
        
        # Loop though all the legs
        for index, legs in enumerate(legs_tracked.people):
            position = legs.pose.position
            legs_point = np.array([position.x,position.y]) #,position.z])
            match_distance = self.calc_distance(face_point_fixed,legs_point)
            if match_distance < best_match_distance:
                best_match_distance = match_distance
                best_match_index = index
        
        # If its close enough, then remember those legs
        if best_match_index is not None:
            self.person_tracked = legs_tracked.people[best_match_index]
            rospy.loginfo(rospy.get_caller_id() + " Matched Leg ID: %s", str(self.person_tracked.id))
            rospy.loginfo(rospy.get_caller_id() + " Matched distance: %s", str(best_match_distance))
        self.face_tracked = msg
        
        
    def legs_callback(self, msg):
        """ Legs topic callback
        Update tracking of legs and set goal to person tracked
        """
        
        target_tracked = None
        for index, legs in enumerate(msg.people):
            if legs.id == self.person_tracked.id:
                target_tracked = legs
                break
        
        if target_tracked is not None:
            goal = MoveBaseGoal()
            goal.target_pose.pose = target_tracked.pose
            goal.target_pose.header = msg.header
            
            pup_goal = PoseStamped()
            pup_goal.pose = target_tracked.pose
            pup_goal.header = msg.header
            
            self.goal_pup.publish(pup_goal)
#             print(goal)
#             sac = SimpleActionClient('move_base', MoveBaseAction)
#             sac.send_goal(goal)
#             print(sac.get_result())
        
        self.legs_tracked=msg

    
    def calc_distance(self, face_point,legs_point):
        """ Calulate distance between two points
        Simple L2 norm
        """
        return np.linalg.norm((face_point,legs_point))



In [ ]:
ne = Follower()

In [ ]:
face_tracked = PositionMeasurement()

In [ ]:
legs_tracked = PersonArray()

In [ ]:
person_tracked = Person()

In [ ]:
sac = SimpleActionClient(movebase_ns, MoveBaseAction)

In [ ]:
pup_goal = PoseStamped()

In [ ]:
bar = [[-0.10946398,],
        [ 0.07148078,],
        [ 0.99141724,],
        [ 1.        ]]
bar

In [ ]:
foo = posemath.fromTf(((1,2,3),(0,0,0,1)))
face_frame_to_leg_frame = posemath.toMatrix(foo)

In [ ]:
spam = face_frame_to_leg_frame*bar
print(spam)
woot = spam[:,3][0:3]
print(woot)

In [ ]:
np.linalg.norm((woot,woot))

In [ ]:
legs_tracked.header.

In [ ]:
person_tracked.